In [ ]:
""" model.ipynb """

# import
import torch
import torch.nn as nn
import torch.nn.functional as F

## Channel List

In [ ]:
# 채널 리스트 / steps
channel_list = [128, 128, 128, 128, 64]
steps = 4

## WSConv2d

In [ ]:
""" WSConv2d """

class WSConv2d(nn.Module):
  # 입력 channel 수, 출력 channel 수, kernel 크기, stride(이동폭), padding(경계 처리), scale 보정 계수
  def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, gain=2):
    super().__init__()

    # 기본 Conv2d layer 생성
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    # Scale 계산
    self.scale = (gain / (in_channels * kernel_size ** 2)) ** 0.5

    # Bias를 따로 저장, conv layer에서는 bias 제거
    self.bias = self.conv.bias
    self.conv.bias = None

    # He 초기화 기준 -> weight/bias 초기화
    # conv.weight: 정규 분포 샘플링
    # bias: 모두 0으로 초기화
    nn.init.normal_(self.conv.weight)
    nn.init.zeros_(self.bias)



  # 입력값 * scale -> weight scaling 효과
  def forward(self, x):

    # bias는 channel별로 reshape후 더함
    out = self.conv(x * self.scale) + self.bias.view(1, -1, 1, 1)
    return out

## PixelNorm

In [ ]:
""" PixelNorm """

class PixelNorm(nn.Module):
  def __init__(self):
    super().__init__()
    # eps -> 분모에 사용
    self.eps = 1e-8


  # 각 픽셀마다 벡터의 크기를 1로 정규화
  # sqrt(mean+eps)로 pixel별 norm 산출
  # x를 norm으로 나눠 픽셀 벡터 크기 = 1로 정규화
  def forward(self, x):
    return x / torch.sqrt(torch.mean(x**2, dim=1, keepdim=True) + self.eps)

## UpDownSampling

In [ ]:
""" Up/Down Sampling """

class UpDownSampling(nn.Module):
  def __init__(self, size):
    super().__init__()

    # scale_factor: 배율
    self.size = size


  def forward(self, x):
    # 최근접 보간 - 해상도 전환 시 빠르고 단순한 연산 수행
    return F.interpolate(x, scale_factor=self.size, mode="nearest")

## MinibatchStd

In [ ]:
""" MinibatchStd """

# Batch 내 통계량 추가 -> 서로 다른 샘플 간 변별력 상승
class MinibatchStd(nn.Module):
  def forward(self, x):
    bs, _, h, w = x.size()

    # Channel별 픽셀 표준편차 계산 -> 전체 평균 -> (bsx1xhxw) 크기로 복제
    std = torch.std(x, dim=0).mean().repeat(bs, 1, h, w)

    # 원본 feature map에 std channel을 추가 -> 미세한 차이 학습 가능
    return torch.cat([x, std], dim=1)

## Generator

In [ ]:
""" Generator Block """

# 해상도를 2배씩 늘려 특징을 점진적으로 확장
class GeneratorConvBlock(nn.Module):
  def __init__(self, step, scale_size):
    super().__init__()

    # 2x upsampling layer: 크기를 scale_size배로 증가
    self.up = UpDownSampling(scale_size)

    # 2번의 WSConv + LeakyReLU + PixelNorm 반복
    # 첫 번째 WSConv2d: 채널 수를 이전 단계 -> 현재 단계로 변환
    self.conv1 = WSConv2d(channel_list[step-1], channel_list[step])

    # 두 번째 WSConv2d: 같은 채널 수 유지하며 추가 특징 학습
    self.conv2 = WSConv2d(channel_list[step], channel_list[step])

    # LeakyReLU 적용 -> 학습 안정화
    self.lrelu = nn.LeakyReLU(0.2)

    # PixelNorm: 픽셀 단위 정규화를 통한 학습 안정화
    self.pn = PixelNorm()


  def forward(self, x):
    # Upsampling -> 해상도 x2
    x = self.up(x)

    # 첫 번째 Conv2 -> LeakyReLU -> PixelNorm
    x = self.lrelu(self.conv1(x))
    x = self.pn(x)

    # 두 번째 Conv2 -> LeakyReLU -> PixelNorm
    x = self.lrelu(self.conv2(x))
    x = self.pn(x)

    return x


""" Generator Structure """
class Generator(nn.Module):
  def __init__(self, steps):
    super().__init__()
    self.steps = steps

    # --- 초기 블록: 4x4 ---
    # PixelNorm: 입력 z 정규화
    # ConvTranspose2d: 채널 list[0] -> 채널_list[0], kernel 4x4 -> 4x4 map 생성
    # LeakyReLU -> WSConv2d -> LeakyReLU -> PixelNorm
    self.init = nn.Sequential(
        PixelNorm(),
        nn.ConvTranspose2d(channel_list[0], channel_list[0], 4, 1, 0),
        nn.LeakyReLU(0.2),
        WSConv2d(channel_list[0], channel_list[0]),
        nn.LeakyReLU(0.2),
        PixelNorm()
    )

    # --- Progressive Block ---
    # init 블록 뒤 -> step=1부터 steps까지 GeneratorConvBlock 쌓기
    # 각 block이 해상도를 2배씩 증가시키며 특징 확장
    self.prog_blocks = nn.ModuleList([self.init] + [GeneratorConvBlock(step, 2) for step in range(1, steps+1)])

    # --- toRGB layer ---
    # 마지막 feature map을 RGB 이미지로 변환 & kernel 크기 1x1로 channel 변환만 수행
    self.toRGB = WSConv2d(channel_list[steps], 3, kernel_size=1, stride=1, padding=0)


  def forward(self, z):
    out = z

    # 초기 4x4 block
    out = self.prog_blocks[0](out)

    # 점진적 해상도 증가 블록 순차 적용(해상도 x2 -> feature map 확장)
    for block in self.prog_blocks[1:]:
      out = block(out)

    # toRGB: 마지막 feature map을 RGB 이미지로 변환
    return self.toRGB(out)

## Discriminator

In [ ]:
""" Discriminator Block """

# 해상도를 단계별로 줄이며 특징 추출
class DiscriminatorConvBlock(nn.Module):
    def __init__(self, step):
      super().__init__()

      # ---각 해상도별 fromRGB layer와 block을 역순으로 쌓음---
      # 첫 번째 WSConv2d: 같은 채널 수 유지하며 nonlinear activation 전 특징 추출
      self.conv1 = WSConv2d(channel_list[step], channel_list[step])

      # 두 번째 WSConv2d: 이전 단계 채널 수로 줄이면서 세밀한 특징 학습
      self.conv2 = WSConv2d(channel_list[step], channel_list[step-1])

      # AvgPool: 해상도 절반으로 줄이는 downsampling
      self.down = nn.AvgPool2d(2,2)

      # LeakyReLU 적용 -> 학습 안정화
      self.lrelu = nn.LeakyReLU(0.2)

    def forward(self,x):
      # conv1 -> LeakyReLU
      x = self.lrelu(self.conv1(x))

      # conv2 -> LeakyReLU
      x = self.lrelu(self.conv2(x))

      # 해상도 절반으로 축소
      return self.down(x)


""" Discriminator Structure """
class Discriminator(nn.Module):
    def __init__(self, steps):
      super().__init__()
      self.steps=steps

      # 각 해상도별 fromRGB layer를 list 순서대로 저장
      self.fromrgb_layers = nn.ModuleList()

      # 단계별 Conv block을 순서대로 저장
      self.prog_blocks = nn.ModuleList()

      # 높은 해상도 -> 낮은 해상도 순으로 layer 구성
      for s in range(steps, 0, -1):

        # RGB 이미지를 channel_list[s] 만큼의 feature map으로 변환
        self.fromrgb_layers.append(WSConv2d(3, channel_list[s], 1, 1, 0))

        # 해당 해상도 단계의 ConvBlock 추가
        self.prog_blocks.append(DiscriminatorConvBlock(s))

      # 최종 해상도용 fromRGB layer
      self.fromrgb_layers.append(WSConv2d(3,channel_list[0], 1, 1, 0))

      # 최종 Discriminator block: Minibatch -> 3x3 conv -> 4x4 conv -> 1x1 conv -> Sigmoid
      self.prog_blocks.append(nn.Sequential(
          # 배치 표준편차 channel 추가
          MinibatchStd(),
          WSConv2d(channel_list[0]+1, channel_list[0], 3, 1, 1),
          nn.LeakyReLU(0.2),
          WSConv2d(channel_list[0], channel_list[0], 4, 1, 0),
          nn.LeakyReLU(0.2),
          # 최종 실수 scalar값 출력
          WSConv2d(channel_list[0], 1, 1, 1, 0),
          # 0~1 확률값으로 변환
          nn.Sigmoid()
      ))

      # 추가 downsample layer 및 활성화
      self.down = nn.AvgPool2d(2,2)
      self.lrelu = nn.LeakyReLU(0.2)


    # fade-in
    def fade_in(self, alpha, down, cur):
      # down: 이전 해상도의 특징 / cur: 현재 해상도의 특징 / alpha 비율로 보간 -> 점진적 성장 학습 안정화
      return alpha*cur + (1-alpha)*down


    def forward(self, x, alpha):
      # 현재 해상도에서부터 처리 시작: fromRGB -> LeakyReLU
      out = self.lrelu(self.fromrgb_layers[0](x))

      # 현재 단계 = 0인 경우 -> 최종 판별 block으로 이동
      if self.steps==0:
          return self.prog_blocks[-1](out).view(out.size(0), -1)

      # 이전 해상도용 특징: Downsampling -> fromRGB -> LeakyReLU
      down = self.lrelu(self.fromrgb_layers[1](self.down(x)))

      # 현재 해상도 ConvBlock 적용
      out = self.prog_blocks[0](out)

      # fade-in 보간 적용
      out = self.fade_in(alpha, down, out)

      # 남아있는 모든 ConvBlock 연쇄 적용
      for i in range(1, self.steps+1):
        out = self.prog_blocks[i](out)

      # 최종 Discriminator 출력
      return out.view(out.size(0), -1)